# Introduction to nmap

[Nmap ("Network Mapper")](https://nmap.org/) is a [free and open source](https://svn.nmap.org/nmap/COPYING) utility for network discovery and security auditing.  
There is a [Python package](https://pypi.python.org/pypi/python-nmap) called [python-nmap](https://xael.org/pages/python-nmap-en.html) that helps in using the nmap port scanner.  
The documentation for nmap can [be found here](https://nmap.org/docs.html), and the Github mirror of the code repository [is here](https://github.com/nmap/nmap).  
For this lesson to work, you need to have at least one port open, preferably 2 or more.

In [1]:
import nmap
# https://github.com/python/cpython/blob/2.7/Lib/inspect.py
import inspect

### Let's begin with a quick examination:

In [2]:
nm = nmap.PortScanner()
nm

In [3]:
# https://docs.python.org/2/library/inspect.html#types-and-members
inspect.getmembers(nm)

[('_PortScanner__process', None),
 ('__class__', nmap.nmap.PortScanner),
 ('__delattr__',
  <method-wrapper '__delattr__' of PortScanner object at 0x10aa2bd50>),
 ('__dict__',
  {'_PortScanner__process': None,
   '_nmap_last_output': u'\nNmap version 7.60 ( https://nmap.org )\nPlatform: x86_64-apple-darwin17.0.0\nCompiled with: liblua-5.3.3 openssl-1.0.2l nmap-libssh2-1.8.0 libz-1.2.8 nmap-libpcre-7.6 nmap-libpcap-1.7.3 nmap-libdnet-1.12 ipv6\nCompiled without:\nAvailable nsock engines: kqueue poll select\n',
   '_nmap_path': 'nmap',
   '_nmap_subversion_number': 60,
   '_nmap_version_number': 7,
   '_scan_result': {}}),
 ('__doc__', '\n    PortScanner class allows to use nmap from python\n\n    '),
 ('__format__', <function __format__>),
 ('__getattribute__',
  <method-wrapper '__getattribute__' of PortScanner object at 0x10aa2bd50>),
 ('__getitem__',
  <bound method PortScanner.__getitem__ of <nmap.nmap.PortScanner object at 0x10aa2bd50>>),
 ('__hash__',
  <method-wrapper '__hash__' 

### Now we can move on to a demonstration:

In [4]:
nm.scan('127.0.0.1', '21-443')

{'nmap': {'command_line': 'nmap -oX - -p 21-443 -sV 127.0.0.1',
  'scaninfo': {'tcp': {'method': 'connect', 'services': '21-443'}},
  'scanstats': {'downhosts': '0',
   'elapsed': '1.50',
   'timestr': 'Wed Feb 14 17:33:03 2018',
   'totalhosts': '1',
   'uphosts': '1'}},
 'scan': {'127.0.0.1': {'addresses': {'ipv4': '127.0.0.1'},
   'hostnames': [{'name': 'localhost', 'type': 'PTR'}],
   'status': {'reason': 'conn-refused', 'state': 'up'},
   'tcp': {22: {'conf': '10',
     'cpe': 'cpe:/a:openbsd:openssh:7.6',
     'extrainfo': 'protocol 2.0',
     'name': 'ssh',
     'product': 'OpenSSH',
     'reason': 'syn-ack',
     'state': 'open',
     'version': '7.6'}},
   'vendor': {}}}}

In [5]:
nm.command_line()

'nmap -oX - -p 21-443 -sV 127.0.0.1'

In [6]:
nm.scaninfo()

{'tcp': {'method': 'connect', 'services': '21-443'}}

In [7]:
nm.all_hosts()

['127.0.0.1']

In [8]:
nm['127.0.0.1'].hostname()

'localhost'

In [9]:
nm['127.0.0.1'].state()

'up'

In [10]:
nm['127.0.0.1'].all_protocols()

['tcp']

In [11]:
nm['127.0.0.1']['tcp'].keys()

[22]

In [12]:
nm['127.0.0.1'].has_tcp(22)

True

In [13]:
nm['127.0.0.1'].has_tcp(21)

False

In [14]:
nm['127.0.0.1']['tcp'][22]

{'conf': '10',
 'cpe': 'cpe:/a:openbsd:openssh:7.6',
 'extrainfo': 'protocol 2.0',
 'name': 'ssh',
 'product': 'OpenSSH',
 'reason': 'syn-ack',
 'state': 'open',
 'version': '7.6'}

In [15]:
nm['127.0.0.1'].tcp(22)

{'conf': '10',
 'cpe': 'cpe:/a:openbsd:openssh:7.6',
 'extrainfo': 'protocol 2.0',
 'name': 'ssh',
 'product': 'OpenSSH',
 'reason': 'syn-ack',
 'state': 'open',
 'version': '7.6'}

In [16]:
nm['127.0.0.1']['tcp'][22]['state']

'open'

### In this section we will write some short scripts:

The following scripts [from the documentation](https://xael.org/pages/python-nmap-en.html) will print out selected information gathered during a port scan.

In [17]:
for host in nm.all_hosts():
    print '-'*80
    print 'Host : %s (%s)' % (host, nm[host].hostname())
    print 'State : %s' % nm[host].state()
    for protocol in nm[host].all_protocols():
        print '-'*30
        print 'Protocol : %s' % protocol
        lport = nm[host][protocol].keys()
        lport.sort()
        for port in lport:
            print 'Port : %s\tState : %s' % (port, nm[host][protocol][port]['state'])

--------------------------------------------------------------------------------
Host : 127.0.0.1 (localhost)
State : up
------------------------------
Protocol : tcp
Port : 22	State : open


In [18]:
print nm.csv()

host;hostname;hostname_type;protocol;port;name;state;product;extrainfo;reason;version;conf;cpe
127.0.0.1;localhost;PTR;tcp;22;ssh;open;OpenSSH;protocol 2.0;syn-ack;7.6;10;cpe:/a:openbsd:openssh:7.6



In [19]:
# The following line of code will take 192 seconds to run (8 seconds for 24 ports).
# It was time to get more coffee anyway.
# Then you can read about port scanning techniques.
# https://nmap.org/book/man-port-scanning-techniques.html
nm.scan(hosts='192.168.1.0/24', arguments='-n -sP -PE -PA21,23,80,3389')

{'nmap': {'command_line': 'nmap -oX - -n -sP -PE -PA21,23,80,3389 192.168.1.0/24',
  'scaninfo': {'warning': [u'Warning:  You are not root -- using TCP pingscan rather than ICMP\n']},
  'scanstats': {'downhosts': '256',
   'elapsed': '206.70',
   'timestr': 'Wed Feb 14 17:36:29 2018',
   'totalhosts': '256',
   'uphosts': '0'}},
 'scan': {}}

In [20]:
hosts_list = [(x, nm[x]['status']['state']) for x in nm.all_hosts()]
for host, status in hosts_list:
    print '{0}:{1}'.host

### Nmap can also perform operations asynchronously:

In [21]:
nma = nmap.PortScannerAsync()
nma

In [22]:
def callback_result(host, scan_result):
    print '-'*30
    print host, scan_result

In [23]:
# Remember earlier when you had to wait for 192 seconds?
# Now the wait is longer.
# Try to figure out how to speed it up using the docs you read earlier.
# Not just the page that you linked to. Try harder.
nma.scan(hosts='192.168.1.0/30', arguments='-sP', callback=callback_result)
while nma.still_scanning():
    print "Waiting >>>"
    nma.wait(2)

Waiting >>>
Waiting >>>
Waiting >>>
------------------------------
192.168.1.0 {'nmap': {'scanstats': {'uphosts': '0', 'timestr': 'Wed Feb 14 17:36:35 2018', 'downhosts': '1', 'totalhosts': '1', 'elapsed': '3.01'}, 'scaninfo': {}, 'command_line': 'nmap -oX - -sP 192.168.1.0'}, 'scan': {}}
Waiting >>>
Waiting >>>
------------------------------
192.168.1.1 {'nmap': {'scanstats': {'uphosts': '0', 'timestr': 'Wed Feb 14 17:36:38 2018', 'downhosts': '1', 'totalhosts': '1', 'elapsed': '3.01'}, 'scaninfo': {}, 'command_line': 'nmap -oX - -sP 192.168.1.1'}, 'scan': {}}
Waiting >>>
------------------------------
192.168.1.2 {'nmap': {'scanstats': {'uphosts': '0', 'timestr': 'Wed Feb 14 17:36:41 2018', 'downhosts': '1', 'totalhosts': '1', 'elapsed': '3.01'}, 'scaninfo': {}, 'command_line': 'nmap -oX - -sP 192.168.1.2'}, 'scan': {}}
Waiting >>>
Waiting >>>
------------------------------
192.168.1.3 {'nmap': {'scanstats': {'uphosts': '0', 'timestr': 'Wed Feb 14 17:36:44 2018', 'downhosts': '1', 't

In [24]:
nm = nmap.PortScannerYield()
for progressive_result in nm.scan('127.0.0.1/24', '21-25'):
    print progressive_result

('127.0.0.0', {'nmap': {'scanstats': {'uphosts': '0', 'timestr': 'Wed Feb 14 17:36:48 2018', 'downhosts': '1', 'totalhosts': '1', 'elapsed': '3.25'}, 'scaninfo': {'tcp': {'services': '21-25', 'method': 'connect'}}, 'command_line': 'nmap -oX - -p 21-25 -sV 127.0.0.0'}, 'scan': {}})
('127.0.0.1', {'nmap': {'scanstats': {'uphosts': '1', 'timestr': 'Wed Feb 14 17:36:48 2018', 'downhosts': '0', 'totalhosts': '1', 'elapsed': '0.35'}, 'scaninfo': {'tcp': {'services': '21-25', 'method': 'connect'}}, 'command_line': 'nmap -oX - -p 21-25 -sV 127.0.0.1'}, 'scan': {'127.0.0.1': {'status': {'state': 'up', 'reason': 'conn-refused'}, 'hostnames': [{'type': 'PTR', 'name': 'localhost'}], 'vendor': {}, 'addresses': {'ipv4': '127.0.0.1'}, 'tcp': {24: {'product': '', 'state': 'closed', 'version': '', 'name': 'priv-mail', 'conf': '3', 'extrainfo': '', 'reason': 'conn-refused', 'cpe': ''}, 25: {'product': '', 'state': 'closed', 'version': '', 'name': 'smtp', 'conf': '3', 'extrainfo': '', 'reason': 'conn-ref

('127.0.0.121', {'nmap': {'scanstats': {'uphosts': '0', 'timestr': 'Wed Feb 14 17:38:11 2018', 'downhosts': '1', 'totalhosts': '1', 'elapsed': '3.27'}, 'scaninfo': {'tcp': {'services': '21-25', 'method': 'connect'}}, 'command_line': 'nmap -oX - -p 21-25 -sV 127.0.0.121'}, 'scan': {}})
('127.0.0.122', {'nmap': {'scanstats': {'uphosts': '0', 'timestr': 'Wed Feb 14 17:38:14 2018', 'downhosts': '1', 'totalhosts': '1', 'elapsed': '3.28'}, 'scaninfo': {'tcp': {'services': '21-25', 'method': 'connect'}}, 'command_line': 'nmap -oX - -p 21-25 -sV 127.0.0.122'}, 'scan': {}})
('127.0.0.123', {'nmap': {'scanstats': {'uphosts': '0', 'timestr': 'Wed Feb 14 17:38:17 2018', 'downhosts': '1', 'totalhosts': '1', 'elapsed': '3.28'}, 'scaninfo': {'tcp': {'services': '21-25', 'method': 'connect'}}, 'command_line': 'nmap -oX - -p 21-25 -sV 127.0.0.123'}, 'scan': {}})
('127.0.0.124', {'nmap': {'scanstats': {'uphosts': '0', 'timestr': 'Wed Feb 14 17:38:21 2018', 'downhosts': '1', 'totalhosts': '1', 'elapsed':

('127.0.0.148', {'nmap': {'scanstats': {'uphosts': '0', 'timestr': 'Wed Feb 14 17:39:47 2018', 'downhosts': '1', 'totalhosts': '1', 'elapsed': '3.27'}, 'scaninfo': {'tcp': {'services': '21-25', 'method': 'connect'}}, 'command_line': 'nmap -oX - -p 21-25 -sV 127.0.0.148'}, 'scan': {}})
('127.0.0.149', {'nmap': {'scanstats': {'uphosts': '0', 'timestr': 'Wed Feb 14 17:39:50 2018', 'downhosts': '1', 'totalhosts': '1', 'elapsed': '3.28'}, 'scaninfo': {'tcp': {'services': '21-25', 'method': 'connect'}}, 'command_line': 'nmap -oX - -p 21-25 -sV 127.0.0.149'}, 'scan': {}})
('127.0.0.15', {'nmap': {'scanstats': {'uphosts': '0', 'timestr': 'Wed Feb 14 17:39:54 2018', 'downhosts': '1', 'totalhosts': '1', 'elapsed': '3.26'}, 'scaninfo': {'tcp': {'services': '21-25', 'method': 'connect'}}, 'command_line': 'nmap -oX - -p 21-25 -sV 127.0.0.15'}, 'scan': {}})
('127.0.0.150', {'nmap': {'scanstats': {'uphosts': '0', 'timestr': 'Wed Feb 14 17:39:57 2018', 'downhosts': '1', 'totalhosts': '1', 'elapsed': '

('127.0.0.174', {'nmap': {'scanstats': {'uphosts': '0', 'timestr': 'Wed Feb 14 17:41:23 2018', 'downhosts': '1', 'totalhosts': '1', 'elapsed': '3.28'}, 'scaninfo': {'tcp': {'services': '21-25', 'method': 'connect'}}, 'command_line': 'nmap -oX - -p 21-25 -sV 127.0.0.174'}, 'scan': {}})
('127.0.0.175', {'nmap': {'scanstats': {'uphosts': '0', 'timestr': 'Wed Feb 14 17:41:27 2018', 'downhosts': '1', 'totalhosts': '1', 'elapsed': '3.28'}, 'scaninfo': {'tcp': {'services': '21-25', 'method': 'connect'}}, 'command_line': 'nmap -oX - -p 21-25 -sV 127.0.0.175'}, 'scan': {}})
('127.0.0.176', {'nmap': {'scanstats': {'uphosts': '0', 'timestr': 'Wed Feb 14 17:41:30 2018', 'downhosts': '1', 'totalhosts': '1', 'elapsed': '3.27'}, 'scaninfo': {'tcp': {'services': '21-25', 'method': 'connect'}}, 'command_line': 'nmap -oX - -p 21-25 -sV 127.0.0.176'}, 'scan': {}})
('127.0.0.177', {'nmap': {'scanstats': {'uphosts': '0', 'timestr': 'Wed Feb 14 17:41:33 2018', 'downhosts': '1', 'totalhosts': '1', 'elapsed':

('127.0.0.20', {'nmap': {'scanstats': {'uphosts': '0', 'timestr': 'Wed Feb 14 17:43:00 2018', 'downhosts': '1', 'totalhosts': '1', 'elapsed': '3.28'}, 'scaninfo': {'tcp': {'services': '21-25', 'method': 'connect'}}, 'command_line': 'nmap -oX - -p 21-25 -sV 127.0.0.20'}, 'scan': {}})
('127.0.0.200', {'nmap': {'scanstats': {'uphosts': '0', 'timestr': 'Wed Feb 14 17:43:03 2018', 'downhosts': '1', 'totalhosts': '1', 'elapsed': '3.26'}, 'scaninfo': {'tcp': {'services': '21-25', 'method': 'connect'}}, 'command_line': 'nmap -oX - -p 21-25 -sV 127.0.0.200'}, 'scan': {}})
('127.0.0.201', {'nmap': {'scanstats': {'uphosts': '0', 'timestr': 'Wed Feb 14 17:43:06 2018', 'downhosts': '1', 'totalhosts': '1', 'elapsed': '3.27'}, 'scaninfo': {'tcp': {'services': '21-25', 'method': 'connect'}}, 'command_line': 'nmap -oX - -p 21-25 -sV 127.0.0.201'}, 'scan': {}})
('127.0.0.202', {'nmap': {'scanstats': {'uphosts': '0', 'timestr': 'Wed Feb 14 17:43:10 2018', 'downhosts': '1', 'totalhosts': '1', 'elapsed': '

('127.0.0.226', {'nmap': {'scanstats': {'uphosts': '0', 'timestr': 'Wed Feb 14 17:44:36 2018', 'downhosts': '1', 'totalhosts': '1', 'elapsed': '3.26'}, 'scaninfo': {'tcp': {'services': '21-25', 'method': 'connect'}}, 'command_line': 'nmap -oX - -p 21-25 -sV 127.0.0.226'}, 'scan': {}})
('127.0.0.227', {'nmap': {'scanstats': {'uphosts': '0', 'timestr': 'Wed Feb 14 17:44:39 2018', 'downhosts': '1', 'totalhosts': '1', 'elapsed': '3.28'}, 'scaninfo': {'tcp': {'services': '21-25', 'method': 'connect'}}, 'command_line': 'nmap -oX - -p 21-25 -sV 127.0.0.227'}, 'scan': {}})
('127.0.0.228', {'nmap': {'scanstats': {'uphosts': '0', 'timestr': 'Wed Feb 14 17:44:43 2018', 'downhosts': '1', 'totalhosts': '1', 'elapsed': '3.28'}, 'scaninfo': {'tcp': {'services': '21-25', 'method': 'connect'}}, 'command_line': 'nmap -oX - -p 21-25 -sV 127.0.0.228'}, 'scan': {}})
('127.0.0.229', {'nmap': {'scanstats': {'uphosts': '0', 'timestr': 'Wed Feb 14 17:44:46 2018', 'downhosts': '1', 'totalhosts': '1', 'elapsed':

('127.0.0.252', {'nmap': {'scanstats': {'uphosts': '0', 'timestr': 'Wed Feb 14 17:46:12 2018', 'downhosts': '1', 'totalhosts': '1', 'elapsed': '3.28'}, 'scaninfo': {'tcp': {'services': '21-25', 'method': 'connect'}}, 'command_line': 'nmap -oX - -p 21-25 -sV 127.0.0.252'}, 'scan': {}})
('127.0.0.253', {'nmap': {'scanstats': {'uphosts': '0', 'timestr': 'Wed Feb 14 17:46:16 2018', 'downhosts': '1', 'totalhosts': '1', 'elapsed': '3.28'}, 'scaninfo': {'tcp': {'services': '21-25', 'method': 'connect'}}, 'command_line': 'nmap -oX - -p 21-25 -sV 127.0.0.253'}, 'scan': {}})
('127.0.0.254', {'nmap': {'scanstats': {'uphosts': '0', 'timestr': 'Wed Feb 14 17:46:19 2018', 'downhosts': '1', 'totalhosts': '1', 'elapsed': '3.27'}, 'scaninfo': {'tcp': {'services': '21-25', 'method': 'connect'}}, 'command_line': 'nmap -oX - -p 21-25 -sV 127.0.0.254'}, 'scan': {}})
('127.0.0.255', {'nmap': {'scanstats': {'uphosts': '0', 'timestr': 'Wed Feb 14 17:46:22 2018', 'downhosts': '1', 'totalhosts': '1', 'elapsed':

('127.0.0.49', {'nmap': {'scanstats': {'uphosts': '0', 'timestr': 'Wed Feb 14 17:47:49 2018', 'downhosts': '1', 'totalhosts': '1', 'elapsed': '3.28'}, 'scaninfo': {'tcp': {'services': '21-25', 'method': 'connect'}}, 'command_line': 'nmap -oX - -p 21-25 -sV 127.0.0.49'}, 'scan': {}})
('127.0.0.5', {'nmap': {'scanstats': {'uphosts': '0', 'timestr': 'Wed Feb 14 17:47:52 2018', 'downhosts': '1', 'totalhosts': '1', 'elapsed': '3.27'}, 'scaninfo': {'tcp': {'services': '21-25', 'method': 'connect'}}, 'command_line': 'nmap -oX - -p 21-25 -sV 127.0.0.5'}, 'scan': {}})
('127.0.0.50', {'nmap': {'scanstats': {'uphosts': '0', 'timestr': 'Wed Feb 14 17:47:55 2018', 'downhosts': '1', 'totalhosts': '1', 'elapsed': '3.28'}, 'scaninfo': {'tcp': {'services': '21-25', 'method': 'connect'}}, 'command_line': 'nmap -oX - -p 21-25 -sV 127.0.0.50'}, 'scan': {}})
('127.0.0.51', {'nmap': {'scanstats': {'uphosts': '0', 'timestr': 'Wed Feb 14 17:47:59 2018', 'downhosts': '1', 'totalhosts': '1', 'elapsed': '3.26'},

('127.0.0.75', {'nmap': {'scanstats': {'uphosts': '0', 'timestr': 'Wed Feb 14 17:49:25 2018', 'downhosts': '1', 'totalhosts': '1', 'elapsed': '3.29'}, 'scaninfo': {'tcp': {'services': '21-25', 'method': 'connect'}}, 'command_line': 'nmap -oX - -p 21-25 -sV 127.0.0.75'}, 'scan': {}})
('127.0.0.76', {'nmap': {'scanstats': {'uphosts': '0', 'timestr': 'Wed Feb 14 17:49:28 2018', 'downhosts': '1', 'totalhosts': '1', 'elapsed': '3.28'}, 'scaninfo': {'tcp': {'services': '21-25', 'method': 'connect'}}, 'command_line': 'nmap -oX - -p 21-25 -sV 127.0.0.76'}, 'scan': {}})
('127.0.0.77', {'nmap': {'scanstats': {'uphosts': '0', 'timestr': 'Wed Feb 14 17:49:32 2018', 'downhosts': '1', 'totalhosts': '1', 'elapsed': '3.28'}, 'scaninfo': {'tcp': {'services': '21-25', 'method': 'connect'}}, 'command_line': 'nmap -oX - -p 21-25 -sV 127.0.0.77'}, 'scan': {}})
('127.0.0.78', {'nmap': {'scanstats': {'uphosts': '0', 'timestr': 'Wed Feb 14 17:49:35 2018', 'downhosts': '1', 'totalhosts': '1', 'elapsed': '3.27'